# Modulos

In [ ]:
import os
import zipfile
import io
import pandas as pd
from pathlib import Path
from datetime import datetime
import pypdf
import docx
import warnings

In [ ]:
# Suprimir advertencias de librerías para mantener la consola limpia
warnings.filterwarnings("ignore")

# ==========================================
# CONFIGURACIÓN
# ==========================================
RUTA_BASE = r"\\Servernas\AYC2\(01. ASEGURAMIENTO)\01. ASEGURAMIENTO\01. REGIMEN SUBSIDIADO\MUNICIPIOS 2009"
ID_USUARIO = "9657820"
ARCHIVO_SALIDA = Path.home() / "Downloads" / f"resultados_busqueda_{ID_USUARIO}.txt"

# ==========================================
# FUNCIONES DE LECTURA ESPECIALIZADAS
# ==========================================

def leer_texto(ruta_o_buffer):
    """Lee archivos de texto plano (.txt, .csv, .log)."""
    try:
        if isinstance(ruta_o_buffer, (str, Path)):
            with open(ruta_o_buffer, 'r', encoding='utf-8', errors='ignore') as f:
                return f.read()
        else:
            # Es un archivo en memoria (dentro de un zip)
            return ruta_o_buffer.read().decode('utf-8', errors='ignore')
    except Exception:
        return ""

def leer_excel(ruta_o_buffer, extension):
    """Lee archivos Excel y convierte todo a string para buscar."""
    contenido = ""
    engine = 'openpyxl' if 'xlsx' in extension else 'xlrd'
    try:
        # Solo leemos celdas como string para evitar problemas de tipos
        dfs = pd.read_excel(ruta_o_buffer, sheet_name=None, engine=engine, dtype=str)
        for hoja, df in dfs.items():
            contenido += df.to_string() + " "
        return contenido
    except Exception:
        return ""

def leer_word(ruta_o_buffer):
    """Lee archivos .docx (Word moderno)."""
    contenido = ""
    try:
        doc = docx.Document(ruta_o_buffer)
        for para in doc.paragraphs:
            contenido += para.text + "\n"
        # También buscar en tablas dentro del Word
        for table in doc.tables:
            for row in table.rows:
                for cell in row.cells:
                    contenido += cell.text + " "
        return contenido
    except Exception:
        return ""

def leer_pdf(ruta_o_buffer):
    """Lee archivos PDF extrayendo texto."""
    contenido = ""
    try:
        pdf_reader = pypdf.PdfReader(ruta_o_buffer)
        for page in pdf_reader.pages:
            texto = page.extract_text()
            if texto:
                contenido += texto + "\n"
        return contenido
    except Exception:
        return ""

# ==========================================
# LÓGICA DE BÚSQUEDA Y RECURSIVIDAD ZIP
# ==========================================

def procesar_archivo(nombre_archivo, objeto_archivo, extension, ruta_visual):
    """
    Orquesta la lectura según la extensión.
    :param nombre_archivo: Nombre del archivo.
    :param objeto_archivo: Ruta (str) o BytesIO (memoria si viene de zip).
    :param extension: Extensión del archivo.
    :param ruta_visual: Ruta para mostrar en el log/resultado.
    :return: True si encontró el ID, False si no.
    """
    texto_extraido = ""
    ext = extension.lower()

    if ext in ['.xlsx', '.xls']:
        texto_extraido = leer_excel(objeto_archivo, ext)
    elif ext in ['.txt', '.csv', '.log', '.xml']:
        texto_extraido = leer_texto(objeto_archivo)
    elif ext == '.docx':
        texto_extraido = leer_word(objeto_archivo)
    elif ext == '.pdf':
        texto_extraido = leer_pdf(objeto_archivo)
    
    # Validar si el ID está en el texto extraído
    if ID_USUARIO in texto_extraido:
        print(f" [!] ENCONTRADO EN: {ruta_visual}")
        return True
    return False

def buscar_recursivo_zip(archivo_zip, ruta_padre):
    """
    Busca dentro de un zip, y si encuentra otro zip, se llama a sí misma.
    Maneja los archivos en memoria RAM para no escribir en disco.
    """
    hallazgos = []
    try:
        with zipfile.ZipFile(archivo_zip, 'r') as z:
            for info_archivo in z.infolist():
                nombre = info_archivo.filename
                ruta_virtual = f"{ruta_padre} >> {nombre}"
                ext = os.path.splitext(nombre)[1].lower()

                # Si es un directorio, ignorar
                if info_archivo.is_dir():
                    continue

                # Leer el archivo en memoria
                with z.open(nombre) as f_in_zip:
                    contenido_bytes = io.BytesIO(f_in_zip.read())
                    
                    # Si es otro ZIP, recursividad
                    if ext == '.zip':
                        hallazgos.extend(buscar_recursivo_zip(contenido_bytes, ruta_virtual))
                    else:
                        # Procesar archivo normal dentro del zip
                        if procesar_archivo(nombre, contenido_bytes, ext, ruta_virtual):
                            hallazgos.append(ruta_virtual)
    except Exception as e:
        # Los zip a veces tienen contraseña o están corruptos
        pass
        
    return hallazgos

# ==========================================
# FUNCIÓN PRINCIPAL
# ==========================================

def main():
    print(f"--- Iniciando Búsqueda Forense ---")
    print(f"Objetivo: {ID_USUARIO}")
    print(f"Ruta: {RUTA_BASE}")
    print(f"Hora inicio: {datetime.now().strftime('%H:%M:%S')}")
    print("-" * 50)

    resultados_finales = []
    archivos_procesados = 0

    for root, dirs, files in os.walk(RUTA_BASE):
        for file in files:
            ruta_completa = os.path.join(root, file)
            extension = os.path.splitext(file)[1].lower()
            archivos_procesados += 1
            
            # Feedback visual simple cada 100 archivos para no saturar consola
            if archivos_procesados % 100 == 0:
                print(f"Escaneando... {archivos_procesados} archivos procesados.", end='\r')

            try:
                encontrado = False
                
                # Caso especial: Archivo ZIP (Raíz)
                if extension == '.zip':
                    hallazgos_zip = buscar_recursivo_zip(ruta_completa, ruta_completa)
                    if hallazgos_zip:
                        resultados_finales.extend(hallazgos_zip)
                
                # Otros archivos normales en disco
                else:
                    if procesar_archivo(file, ruta_completa, extension, ruta_completa):
                        resultados_finales.append(ruta_completa)

            except Exception as e:
                # Errores de permisos o rutas largas
                continue

    # Guardar reporte
    with open(ARCHIVO_SALIDA, 'w', encoding='utf-8') as f:
        f.write(f"REPORTE DE BÚSQUEDA\n")
        f.write(f"ID Buscado: {ID_USUARIO}\n")
        f.write(f"Fecha: {datetime.now()}\n")
        f.write(f"Archivos escaneados: {archivos_procesados}\n")
        f.write(f"Coincidencias encontradas: {len(resultados_finales)}\n")
        f.write("="*60 + "\n\n")
        
        if resultados_finales:
            for res in resultados_finales:
                f.write(f"{res}\n")
        else:
            f.write("No se encontraron coincidencias.\n")

    print(f"\n{'='*50}")
    print(f"Proceso finalizado.")
    print(f"Total encontrados: {len(resultados_finales)}")
    print(f"Reporte guardado en: {ARCHIVO_SALIDA}")

if __name__ == "__main__":
    main()